In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import config
from derma.dataset import Derma
from derma.architecture import InvertedResidual

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models import MobileNetV2
from torchvision import transforms

torch.cuda.is_available()

In [ ]:
net = [1,2,4,3]  # 1: MbV2, 2: MbV2_CA, 3: MbV2_CA_Reduced, 4: MbV2_Reduced
DB_used = ['HAM10000'] # 'HAM10000', 'ISIC2019', 'ISIC_prep'
experiments = [[net[j],DB_used[i]] for i in range(len(DB_used)) for j in range(len(net))]

In [ ]:
def load_test_results(test,DB_used):
    from config import DATASET_DIR, RESULT_DIR
    from derma.dataset import get_samples_weight
    from derma.experiment import test_experiment
    inverted_residual_setting_v0 = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ] #ORIGINAL
    inverted_residual_setting_vT3 = [
            # t, c, n, s
            [1, 16, 1, 1],
            [4, 24, 1, 2],
            [4, 32, 1, 2],
            [4, 64, 1, 2],
            [4, 96, 1, 1],
            [4, 160, 1, 2],
            [4, 320, 1, 1],
        ]
    if test == 1:
        Test = 'MbV2'
        CoordAtt = False
        inverted_residual_setting = inverted_residual_setting_v0
    elif test == 2:
        Test = 'MbV2_CA'
        CoordAtt = True
        inverted_residual_setting = inverted_residual_setting_v0
    elif test == 3:
        Test = 'MbV2_CA_Reduced'
        CoordAtt = True
        inverted_residual_setting = inverted_residual_setting_vT3
    elif test == 4:
        Test = 'MbV2_Reduced'
        CoordAtt = False
        inverted_residual_setting = inverted_residual_setting_vT3

    dataset_dir = os.path.join(DATASET_DIR,'HAM10000','Test')

    if DB_used == 'HAM10000':
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256))
        ])
    elif DB_used == 'HAM_norm':
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    elif DB_used == 'ISIC_prep':
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256))
        ])

    test_set = Derma(dataset_dir,labels=[0,1],transform=transform)
    names = test_set.getnames()
    test_loader = DataLoader(test_set, batch_size=224, num_workers=0, shuffle=False)

    model_dir = os.path.join(RESULT_DIR,'weights',Test,DB_used,'model.pth')
    if CoordAtt:
        model = MobileNetV2(num_classes=2, inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)
    else:
        model = MobileNetV2(num_classes=2, inverted_residual_setting=inverted_residual_setting) # standard MobileNetV2
    if torch.cuda.is_available():
        device = torch.device('cuda')
        model.load_state_dict(torch.load(model_dir))
        model.to(device)
    else:
        model.load_state_dict(torch.load(model_dir),map_location=torch.device('cpu'))

    test_result, test_target, test_predicted = test_experiment(model,test_loader)
    experiment_name = DB_used + '_' + Test
    return experiment_name, test_result, test_target, test_predicted, names



In [ ]:
import pandas as pd
import tqdm
from torch import argmax
pd.options.display.float_format = '{:,.4f}'.format
# metrics
columns = ['Experiment','Sensitivity', 'Specificity', 'Precission', 'Recall', 'Accuracy']
metrics = pd.DataFrame(columns=columns)
# output
columns = ['Image','Target','L_MbV2','L_MbV2CA','L_MbV2P','L_MbV2PCA','L_MbV2_norm','L_MbV2CA_norm','L_MbV2P_norm','L_MbV2PCA_norm']
output_matrix = np.empty([224,10])
output = pd.DataFrame(output_matrix,columns=columns)
for exp in tqdm.tqdm(experiments):
    exp_net = exp[0]
    exp_DB = exp[1]
    if exp_net == 1 or exp_net ==2:
        idx = exp_net
    elif exp_net == 3:
        idx = 4
    elif exp_net == 4:
        idx = 3
    if exp_DB == 'HAM_norm':
        norm = True
    elif (exp_DB == 'HAM10000' or exp_DB == 'ISIC_prep'):
        norm = False 

    experiment_name, test_result, test_target, test_predicted, names = load_test_results(exp_net,exp_DB)

    metrics = pd.concat([metrics,test_result])
    metrics.iloc[-1,0] = experiment_name

    for i in range(224):
        _, name = os.path.split(names[i])
        if idx == 1:
            output.iloc[i,0] = name
            output.iloc[i,1] = int(test_target[i].detach().numpy())
        else:
            if test_target[i].detach().numpy() != output.iloc[i,1]:
                print('Target doesnt match')
                break
        output.iloc[i,1+idx+norm*4] = int(argmax(test_predicted[i]).detach().numpy())

In [ ]:
metrics.to_csv('../results/metrics_test.csv',index=False) # to check everything went right
output.to_csv('../results/output_test.csv',index=False)
output.to_excel('../results/output_test.xlsx',index=False)


In [ ]:
from sklearn.metrics import confusion_matrix

def get_simple_metrics(tp,fn,tn,fp):
    accuracy = (tp + tn) / (tp+fn+tn+fp)
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp+fn)
    precission = tp / (tp+fp)
    recall = tp / (tp+fn)
    return specificity, sensitivity, precission, recall, accuracy

models = ['MbV2   ','MbV2CA ','MbV2P  ','MbV2PCA']
for i in range(4):
    targets = output.iloc[:,1]
    predict = output.iloc[:,2+i]
    tn, fp, fn, tp = confusion_matrix(targets, predict,labels=[0,1]).ravel()
    specificity, sensitivity, precission, recall, accuracy = get_simple_metrics(tp,fn,tn,fp)
    print(models[i],specificity, sensitivity, precission, recall, accuracy)
